## Vision Transformer (ViT)

In this assignment we're going to work with Vision Transformer. We will start to build our own vit model and train it on an image classification task.
The purpose of this homework is for you to get familar with ViT and get prepared for the final project.

In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.transforms import v2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# VIT Implementation

The vision transformer can be seperated into three parts, we will implement each part and combine them in the end.

For the implementation, feel free to experiment different kinds of setup, as long as you use attention as the main computation unit and the ViT can be train to perform the image classification task present later.
You can read about the ViT implement from other libary: https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py and https://github.com/pytorch/vision/blob/main/torchvision/models/vision_transformer.py

## PatchEmbedding
PatchEmbedding is responsible for dividing the input image into non-overlapping patches and projecting them into a specified embedding dimension. It uses a 2D convolution layer with a kernel size and stride equal to the patch size. The output is a sequence of linear embeddings for each patch.

In [3]:
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim):
      super(PatchEmbedding, self).__init__()

      self.image_size = image_size
      self.patch_size = patch_size
      self.in_channels = in_channels
      self.embed_dim = embed_dim
      self.patch_count = (image_size // patch_size) ** 2
      self.embedding_layer = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):

      x = self.embedding_layer(x).flatten(start_dim=2).transpose(1, 2)
      return x

## MultiHeadSelfAttention

This class implements the multi-head self-attention mechanism, which is a key component of the transformer architecture. It consists of multiple attention heads that independently compute scaled dot-product attention on the input embeddings. This allows the model to capture different aspects of the input at different positions. The attention outputs are concatenated and linearly transformed back to the original embedding size.

In [4]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout_rate=0.1):

      super(MultiHeadSelfAttention, self).__init__()
      self.embed_dim = embed_dim
      self.num_heads = num_heads
      self.dropout_rate = dropout_rate
      self.head_dim = embed_dim // num_heads
      self.factor = self.head_dim ** -0.5
      self.query = nn.Linear(embed_dim, embed_dim)
      self.key = nn.Linear(embed_dim, embed_dim)
      self.value = nn.Linear(embed_dim, embed_dim)
      self.projection = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):

      batch_size, sequence_len, embedding_dim = x.shape
      queries = self.query(x)
      keys = self.key(x)
      values = self.value(x)
      queries = queries.view(batch_size, sequence_len, self.num_heads, self.head_dim).transpose(1, 2)
      keys = keys.view(batch_size, sequence_len, self.num_heads, self.head_dim).transpose(1, 2)
      values = values.view(batch_size, sequence_len, self.num_heads, self.head_dim).transpose(1, 2)
      attention_scores = (queries @ keys.transpose(-2, -1)) * self.factor
      attention_weights = F.softmax(attention_scores, dim=-1)
      attention_output = attention_weights @ values
      attention_output = attention_output.transpose(1, 2).contiguous()
      combined_output = attention_output.view(batch_size, sequence_len, embedding_dim)
      output = self.projection(combined_output)
      return output

## TransformerBlock
This class represents a single transformer layer. It includes a multi-head self-attention sublayer followed by a position-wise feed-forward network (MLP). Each sublayer is surrounded by residual connections.
You may also want to use layer normalization or other type of normalization.

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout):
        super(TransformerBlock, self).__init__()

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.dropout = dropout

        self.normalization_1 = nn.LayerNorm(embed_dim)
        self.self_attention = MultiHeadSelfAttention(embed_dim, num_heads)
        self.dropout_layer = nn.Dropout(dropout)
        self.normalization_2 = nn.LayerNorm(embed_dim)
        self.fc1 = nn.Linear(embed_dim, mlp_dim)
        self.activation = nn.GELU()
        self.mlp_dropout_1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(mlp_dim, embed_dim)
        self.mlp_dropout_2 = nn.Dropout(dropout)

    def forward(self, x):

        residual = x
        x = self.normalization_1(x)
        x = self.self_attention(x)
        x = self.dropout_layer(x)
        x = x + residual
        residual = x
        x = self.normalization_2(x)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.mlp_dropout_1(x)
        x = self.fc2(x)
        x = self.mlp_dropout_2(x)
        x = x + residual
        return x

## VisionTransformer:
This is the main class that assembles the entire Vision Transformer architecture. It starts with the PatchEmbedding layer to create patch embeddings from the input image. A special class token is added to the sequence, and positional embeddings are added to both the patch and class tokens. The sequence of patch embeddings is then passed through multiple TransformerBlock layers. The final output is the logits for all classes

In [6]:
class VisionTransformer(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout=0.1):

        super(VisionTransformer, self).__init__()
        self.patch_embedding = PatchEmbedding(image_size, patch_size, in_channels, embed_dim)
        self.position_embedding = nn.Parameter(torch.randn(1, self.patch_embedding.patch_count + 1, embed_dim))
        self.class_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.dropout_layer = nn.Dropout(dropout)
        self.transformer_layers = nn.ModuleList(
            [TransformerBlock(embed_dim, num_heads, mlp_dim, dropout) for k in range(num_layers)]
        )
        self.classifier_head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):

        batch_size = x.size(0)
        patch_embeddings = self.patch_embedding(x)
        class_tokens = self.class_token.expand(batch_size, -1, -1)
        embeddings = torch.cat([class_tokens, patch_embeddings], dim = 1)
        embeddings = embeddings + self.position_embedding
        embeddings = self.dropout_layer(embeddings)
        for layer in self.transformer_layers:
          embeddings = layer(embeddings)
        class_output = embeddings[:, 0]
        output = self.classifier_head(class_output)
        return output

## Let's train the ViT!

We will train the vit to do the image classification with cifar100. Free free to change the optimizer and or add other tricks to improve the training

In [7]:
# image_size = 64
image_size = 32
# patch_size = 8
patch_size = 4
in_channels = 3
embed_dim = 256
num_heads = 8
mlp_dim = 512
num_layers = 6
num_classes = 100
dropout = 0.01
batch_size = 256

In [8]:
model = VisionTransformer(image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout).to(device)
input_tensor = torch.randn(1, in_channels, image_size, image_size).to(device)
output = model(input_tensor)
print(output.shape)

torch.Size([1, 100])


In [9]:
# Load the CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

from torch.utils.data import default_collate
cutmix = v2.CutMix(num_classes = 100)
mixup = v2.MixUp(num_classes = 100)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

def collate_fn(batch):
    return cutmix_or_mixup.forward(*default_collate(batch))

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:01<00:00, 103MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [11]:
# Train the model
num_epochs = 100 # TODO
best_val_acc = 0
for epoch in range(num_epochs):
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)


        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    lr_scheduler.step()

        # TODO Feel free to modify the training loop youself.

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Epoch: {epoch + 1}, Validation Accuracy: {val_acc:.2f}%")

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

Epoch: 1, Validation Accuracy: 16.06%
Epoch: 2, Validation Accuracy: 21.76%
Epoch: 3, Validation Accuracy: 24.78%
Epoch: 4, Validation Accuracy: 28.20%
Epoch: 5, Validation Accuracy: 30.69%
Epoch: 6, Validation Accuracy: 33.10%
Epoch: 7, Validation Accuracy: 35.94%
Epoch: 8, Validation Accuracy: 37.68%
Epoch: 9, Validation Accuracy: 39.22%
Epoch: 10, Validation Accuracy: 39.91%
Epoch: 11, Validation Accuracy: 42.26%
Epoch: 12, Validation Accuracy: 44.76%
Epoch: 13, Validation Accuracy: 44.56%
Epoch: 14, Validation Accuracy: 46.79%
Epoch: 15, Validation Accuracy: 47.90%
Epoch: 16, Validation Accuracy: 48.31%
Epoch: 17, Validation Accuracy: 47.90%
Epoch: 18, Validation Accuracy: 49.69%
Epoch: 19, Validation Accuracy: 49.72%
Epoch: 20, Validation Accuracy: 50.86%
Epoch: 21, Validation Accuracy: 52.07%
Epoch: 22, Validation Accuracy: 51.75%
Epoch: 23, Validation Accuracy: 54.15%
Epoch: 24, Validation Accuracy: 54.38%
Epoch: 25, Validation Accuracy: 53.81%
Epoch: 26, Validation Accuracy: 54

Please submit your best_model.pth with this notebook. And report the best test results you get.